Deep Learning
=============

Assignment 6
------------

After training a skip-gram model in `5_word2vec.ipynb`, the goal of this notebook is to train a LSTM character model over [Text8](http://mattmahoney.net/dc/textdata) data.

In [2]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import os
import numpy as np
import random
import string
import tensorflow as tf
import zipfile
from six.moves import range
from six.moves.urllib.request import urlretrieve

In [3]:
url = 'http://mattmahoney.net/dc/'

def maybe_download(filename, expected_bytes):
  """Download a file if not present, and make sure it's the right size."""
  if not os.path.exists(filename):
    filename, _ = urlretrieve(url + filename, filename)
  statinfo = os.stat(filename)
  if statinfo.st_size == expected_bytes:
    print('Found and verified %s' % filename)
  else:
    print(statinfo.st_size)
    raise Exception(
      'Failed to verify ' + filename + '. Can you get to it with a browser?')
  return filename

filename = maybe_download('text8.zip', 31344016)

Found and verified text8.zip


In [4]:
def read_data(filename):
  f = zipfile.ZipFile(filename)
  for name in f.namelist():
    return tf.compat.as_str(f.read(name))
  f.close()
  
text = read_data(filename)
print('Data size %d' % len(text))

Data size 100000000


Create a small validation set.

In [49]:
valid_size = 1000
valid_text = text[:valid_size]
train_text = text[valid_size:]
train_size = len(train_text)
print(train_size, train_text[:1000])
print(valid_size, valid_text[:64])

99999000 ons anarchists advocate social relations based upon voluntary association of autonomous individuals mutual aid and self governance while anarchism is most easily defined by what it is against anarchists also offer positive visions of what they believe to be a truly free society however ideas about how an anarchist society might work vary considerably especially with respect to economics there is also disagreement about how a free society might be brought about origins and predecessors kropotkin and others argue that before recorded history human society was organized on anarchist principles most anthropologists follow kropotkin and engels in believing that hunter gatherer bands were egalitarian and lacked division of labour accumulated wealth or decreed law and had equal access to resources william godwin anarchists including the the anarchy organisation and rothbard find anarchist attitudes in taoism from ancient china kropotkin found similar ideas in stoic zeno of citium acc

Utility functions to map characters to vocabulary IDs and back.

In [48]:
vocabulary_size = len(string.ascii_lowercase) + 1 # [a-z] + ' '
first_letter = ord(string.ascii_lowercase[0])

def char2id(char):
  if char in string.ascii_lowercase:
    return ord(char) - first_letter + 1
  elif char == ' ':
    return 0
  else:
    print('Unexpected character: %s' % char)
    return 0
  
def id2char(dictid):
  if dictid > 0:
    return chr(dictid + first_letter - 1)
  else:
    return ' '

print(char2id('a'), char2id('z'), char2id(' '), char2id('ï'))
print(id2char(1), id2char(26), id2char(0))

Unexpected character: ï
1 26 0 0
a z  


Function to generate a training batch for the LSTM model.

In [47]:


batch_size=64
num_unrollings=10

class BatchGenerator(object):
  def __init__(self, text, batch_size, num_unrollings):
    self._text = text
    self._text_size = len(text)
    self._batch_size = batch_size
    self._num_unrollings = num_unrollings
    segment = self._text_size // batch_size
    self._cursor = [ offset * segment for offset in range(batch_size)]
    self._last_batch = self._next_batch()
  
  def _next_batch(self):
    """Generate a single batch from the current cursor position in the data."""
    batch = np.zeros(shape=(self._batch_size, vocabulary_size), dtype=np.float)
    for b in range(self._batch_size):
      batch[b, char2id(self._text[self._cursor[b]])] = 1.0
      #print("Character %s in batch %d" % (self._text[self._cursor[b]],b))
      self._cursor[b] = (self._cursor[b] + 1) % self._text_size
    return batch
  
  def next(self):
    """Generate the next array of batches from the data. The array consists of
    the last batch of the previous array, followed by num_unrollings new ones.
    """
    batches = [self._last_batch]
    for step in range(self._num_unrollings):
      batches.append(self._next_batch())
    self._last_batch = batches[-1] # this is where we are adding character from previous batches to subsequent.
    return batches

def characters(probabilities):
  """Turn a 1-hot encoding or a probability distribution over the possible
  characters back into its (most likely) character representation."""
  return [id2char(c) for c in np.argmax(probabilities, 1)]

def batches2string(batches):
  """Convert a sequence of batches back into their (most likely) string
  representation."""
  s = [''] * batches[0].shape[0]
  for b in batches:
    s = [''.join(x) for x in zip(s, characters(b))]
  return s

train_batches = BatchGenerator(train_text, batch_size, num_unrollings)
valid_batches = BatchGenerator(valid_text, 1, 1)

np.set_printoptions(threshold=np.inf) #print everything.

#print(batches2string(train_batches.next())[0])
#print(np.shape(train_batches.next()))

#VD: batch is (11, 64, 27) and consists of:
# - 11 - the last symbol of the previous array, followed by num_unrollings (10) new ones.
# - 64 - is a number of batches, where 1 batch=strings (e.g. 'ons anarchi')
# - 27 vocabulary size (english alphabet + " ")

#print(train_batches.next()[0])
#print(batches2string(train_batches.next()))
#print(batches2string(train_batches.next()))
print(batches2string(valid_batches.next()))
#print(batches2string(valid_batches.next()))


[' a']


In [46]:
def logprob(predictions, labels):
  """Log-probability of the true labels in a predicted batch."""
  predictions[predictions < 1e-10] = 1e-10
  return np.sum(np.multiply(labels, -np.log(predictions))) / labels.shape[0]

def sample_distribution(distribution):
  """Sample one element from a distribution assumed to be an array of normalized
  probabilities.
  """
  r = random.uniform(0, 1)
  s = 0
  for i in range(len(distribution)):
    s += distribution[i]
    if s >= r:
      return i
  return len(distribution) - 1

def sample(prediction):
  """Turn a (column) prediction into 1-hot encoded samples."""
  p = np.zeros(shape=[1, vocabulary_size], dtype=np.float)
  p[0, sample_distribution(prediction[0])] = 1.0
  return p

def random_distribution():
  """Generate a random column of probabilities."""
  b = np.random.uniform(0.0, 1.0, size=[1, vocabulary_size])
  return b/np.sum(b, 1)[:,None]

Simple LSTM Model.

In [20]:
num_nodes = 64 #Number of nodes in NN layers. It is equal to number of batches. I guess otherwise, it won't work.

graph = tf.Graph()
with graph.as_default():
  
  # Parameters:
  # Input gate: input, previous output, and bias.
  ix = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  im = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  ib = tf.Variable(tf.zeros([1, num_nodes]))
  # Forget gate: input, previous output, and bias.
  fx = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  fm = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  fb = tf.Variable(tf.zeros([1, num_nodes]))
  # Memory cell: input, state and bias.                             
  cx = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  cm = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  cb = tf.Variable(tf.zeros([1, num_nodes]))
  # Output gate: input, previous output, and bias.
  ox = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  om = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  ob = tf.Variable(tf.zeros([1, num_nodes]))
  # Variables saving state across unrollings. 
  # VD: afaiu, unrolling is refered to horizon of memory applied to input
  saved_output = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  saved_state = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  # Classifier weights and biases.
  w = tf.Variable(tf.truncated_normal([num_nodes, vocabulary_size], -0.1, 0.1))
  b = tf.Variable(tf.zeros([vocabulary_size]))
  
  # Definition of the cell computation.
  def lstm_cell(i, o, state):
    """Create a LSTM cell. See e.g.: http://arxiv.org/pdf/1402.1128v1.pdf
    Note that in this formulation, we omit the various connections between the
    previous state and the gates."""
    
    #VD let's breakdown operations.
    #1. Input gate. add to current input previous output and bias. Run it through sigmoid.
    #shapes: [batch_size,vocabulary_size]*[vocab_size, num_nodes] + [batch size, num nodes] + [1, num nodes]
    input_gate = tf.sigmoid(tf.matmul(i, ix) + tf.matmul(o, im) + ib) 
    #print("Input shape: %s" % i.get_shape())
    #print("Out shape: %s" % o.get_shape())
    #print("Input gate shape: %s" % input_gate.get_shape())
    
    #2. Forget gate. add to current input previous output and bias. Run it through sigmoid.
    #shapes: [batch_size,vocabulary_size]*[vocab_size, num_nodes] + [batch size, num nodes] + [1, num nodes]
    forget_gate = tf.sigmoid(tf.matmul(i, fx) + tf.matmul(o, fm) + fb)
    
    #3. Update or candidate. add to current input previous output and bias. Run it through sigmoid.
    #shapes: [batch_size,vocabulary_size]*[vocab_size, num_nodes] + [batch size, num nodes] + [1, num nodes]
    update = tf.matmul(i, cx) + tf.matmul(o, cm) + cb
    
    #4. Here "*" is a pointwise multiplication and implemented filtering of previous state using sigmoid within (0,1)
    # also tahn is withing (-1,1)
    state = forget_gate * state + input_gate * tf.tanh(update)
    output_gate = tf.sigmoid(tf.matmul(i, ox) + tf.matmul(o, om) + ob)
    return output_gate * tf.tanh(state), state
    # output=tf.sigmoid(tf.matmul(i, ox) + tf.matmul(o, om) + ob)*
                                        #*tf.tanh(forget_gate * state + input_gate * tf.tanh(update))

  # Input data.
  train_data = list()
  for _ in range(num_unrollings + 1):
    train_data.append(
      tf.placeholder(tf.float32, shape=[batch_size,vocabulary_size]))
  train_inputs = train_data[:num_unrollings]
  train_labels = train_data[1:]  # labels are inputs shifted by one time step.

  # Unrolled LSTM loop.
  outputs = list()
  output = saved_output
  state = saved_state
  for i in train_inputs:
    output, state = lstm_cell(i, output, state)
    outputs.append(output)

  # State saving across unrollings.
  # VD: tf.control_dependecies make sure that input Ops are executed before executing internal graph code. 
  # In this particul state we want to save last output and last state for next batch iteration.
  # VD: tf.Variable.assign assigns a new value to the variable.
  with tf.control_dependencies([saved_output.assign(output),
                                saved_state.assign(state)]):
    # Classifier.
    logits = tf.nn.xw_plus_b(tf.concat(0, outputs), w, b) #VD: should be similar to tf.matmul(x, w) + b 
    loss = tf.reduce_mean(
      tf.nn.softmax_cross_entropy_with_logits(
        logits, tf.concat(0, train_labels)))

  # Optimizer.
  global_step = tf.Variable(0)
  learning_rate = tf.train.exponential_decay(
    10.0, global_step, 5000, 0.1, staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate)
  gradients, v = zip(*optimizer.compute_gradients(loss))
  gradients, _ = tf.clip_by_global_norm(gradients, 1.25) # VD: clipping of the gradients to avoid explosion.
  optimizer = optimizer.apply_gradients(
    zip(gradients, v), global_step=global_step)

  # Predictions.
  train_prediction = tf.nn.softmax(logits)
  
  # Sampling and validation eval: batch 1, no unrolling.
  sample_input = tf.placeholder(tf.float32, shape=[1, vocabulary_size])
  saved_sample_output = tf.Variable(tf.zeros([1, num_nodes]))
  saved_sample_state = tf.Variable(tf.zeros([1, num_nodes]))
    # VD: tf.group creats an op that groups multiple operations.
  reset_sample_state = tf.group(
    saved_sample_output.assign(tf.zeros([1, num_nodes])),
    saved_sample_state.assign(tf.zeros([1, num_nodes])))
  sample_output, sample_state = lstm_cell(
    sample_input, saved_sample_output, saved_sample_state)
  with tf.control_dependencies([saved_sample_output.assign(sample_output),
                                saved_sample_state.assign(sample_state)]):
    sample_prediction = tf.nn.softmax(tf.nn.xw_plus_b(sample_output, w, b))

Tensor("mul_2:0", shape=(64, 64), dtype=float32)
Tensor("add_6:0", shape=(64, 64), dtype=float32)
Tensor("mul_5:0", shape=(64, 64), dtype=float32)
Tensor("add_15:0", shape=(64, 64), dtype=float32)
Tensor("mul_8:0", shape=(64, 64), dtype=float32)
Tensor("add_24:0", shape=(64, 64), dtype=float32)
Tensor("mul_11:0", shape=(64, 64), dtype=float32)
Tensor("add_33:0", shape=(64, 64), dtype=float32)
Tensor("mul_14:0", shape=(64, 64), dtype=float32)
Tensor("add_42:0", shape=(64, 64), dtype=float32)
Tensor("mul_17:0", shape=(64, 64), dtype=float32)
Tensor("add_51:0", shape=(64, 64), dtype=float32)
Tensor("mul_20:0", shape=(64, 64), dtype=float32)
Tensor("add_60:0", shape=(64, 64), dtype=float32)
Tensor("mul_23:0", shape=(64, 64), dtype=float32)
Tensor("add_69:0", shape=(64, 64), dtype=float32)
Tensor("mul_26:0", shape=(64, 64), dtype=float32)
Tensor("add_78:0", shape=(64, 64), dtype=float32)
Tensor("mul_29:0", shape=(64, 64), dtype=float32)
Tensor("add_87:0", shape=(64, 64), dtype=float32)


In [15]:
num_steps = 7001
summary_frequency = 100

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  mean_loss = 0
  for step in range(num_steps):
    batches = train_batches.next()
    feed_dict = dict()
    for i in range(num_unrollings + 1):
      feed_dict[train_data[i]] = batches[i]
    _, l, predictions, lr = session.run(
      [optimizer, loss, train_prediction, learning_rate], feed_dict=feed_dict)
    mean_loss += l
    if step % summary_frequency == 0:
      if step > 0:
        mean_loss = mean_loss / summary_frequency
      # The mean loss is an estimate of the loss over the last few batches.
      print(
        'Average loss at step %d: %f learning rate: %f' % (step, mean_loss, lr))
      mean_loss = 0
      labels = np.concatenate(list(batches)[1:])
      print('Minibatch perplexity: %.2f' % float(
        np.exp(logprob(predictions, labels))))
      if step % (summary_frequency * 10) == 0:
        # Generate some samples.
        print('=' * 80)
        for _ in range(5):
          feed = sample(random_distribution())
          sentence = characters(feed)[0]
          reset_sample_state.run()
          for _ in range(79):
            prediction = sample_prediction.eval({sample_input: feed})
            feed = sample(prediction)
            sentence += characters(feed)[0]
          print(sentence)
        print('=' * 80)
      # Measure validation set perplexity.
      reset_sample_state.run() #VD this is reset previous memory of the LSTM cell. 
                               #It means that we are involving only classifire 
      valid_logprob = 0
      for _ in range(valid_size):
        b = valid_batches.next()
        #VD: below we pass valid set consisting of two chars (e.g. "[' a']")
        predictions = sample_prediction.eval({sample_input: b[0]})
        valid_logprob = valid_logprob + logprob(predictions, b[1])
      print('Validation set perplexity: %.2f' % float(np.exp(
        valid_logprob / valid_size)))

Initialized
Average loss at step 0: 3.298338 learning rate: 10.000000
Minibatch perplexity: 27.07
nhyphnhuelgnvelng abeniea l fegeantwqrcualbktugeenjoq   d lxkmf olgrnnpef afmmx 
dneetaascxcgqpnpczplvqjntliocoamhu aoc rufn mais pdex yutsfwidd wjrxrwphuecn f a
qvvcodka m ovu  aoexedu m s tcsm iepzooddabftzcjnst ip nbjdrtnor ho oqdso   e re
gtqas jd eousewkkomag fuznhhxxc h  civzww l zurat btlrdnexhxn rwnaziketodnemeeua
pxsmr wyt sz wx caym c  ibadyjss p nqbtdsa qwtkhapsmam yezmpczmedgqnqrpft c xrt 
Validation set perplexity: 20.15
Average loss at step 100: 2.594232 learning rate: 10.000000
Minibatch perplexity: 10.75
Validation set perplexity: 10.14
Average loss at step 200: 2.248669 learning rate: 10.000000
Minibatch perplexity: 8.48
Validation set perplexity: 8.39
Average loss at step 300: 2.091511 learning rate: 10.000000
Minibatch perplexity: 7.48
Validation set perplexity: 7.89
Average loss at step 400: 1.994529 learning rate: 10.000000
Minibatch perplexity: 7.49
Validation set per

---
Problem 1
---------

You might have noticed that the definition of the LSTM cell involves 4 matrix multiplications with the input, and 4 matrix multiplications with the output. Simplify the expression by using a single matrix multiply for each, and variables that are 4 times larger.

---

In [8]:
#details here: https://discussions.udacity.com/t/deep-learning-assignment-6-problem-2/191791/19


In [51]:
num_nodes = 64 #Number of nodes in NN layers. It is equal to number of batches. I guess otherwise, it won't work.

graph2 = tf.Graph()
with graph2.as_default():
  
  # Parameters:
  x_weights=tf.Variable(tf.truncated_normal([vocabulary_size, 4*num_nodes], -0.1, 0.1))
  o_weights=tf.Variable(tf.truncated_normal([num_nodes, 4*num_nodes], -0.1, 0.1))
  biases=tf.Variable(tf.zeros([1, 4*num_nodes]))
  
  combined=tf.Variable(batch_size,4*num_nodes)
  #input shape=[batch_size,vocabulary_size])
  
  # Input gate: input, previous output, and bias.
  ix = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  im = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  ib = tf.Variable(tf.zeros([1, num_nodes]))
  # Forget gate: input, previous output, and bias.
  fx = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  fm = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  fb = tf.Variable(tf.zeros([1, num_nodes]))
  # Memory cell: input, state and bias.                             
  cx = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  cm = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  cb = tf.Variable(tf.zeros([1, num_nodes]))
  # Output gate: input, previous output, and bias.
  ox = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  om = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  ob = tf.Variable(tf.zeros([1, num_nodes]))
  # Variables saving state across unrollings. 
  # VD: afaiu, unrolling is refered to horizon of memory applied to input
  saved_output = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  saved_state = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  # Classifier weights and biases.
  w = tf.Variable(tf.truncated_normal([num_nodes, vocabulary_size], -0.1, 0.1))
  b = tf.Variable(tf.zeros([vocabulary_size]))
  
  # Definition of the cell computation.
  def lstm_cell(i, o, state):
    """Create a LSTM cell. See e.g.: http://arxiv.org/pdf/1402.1128v1.pdf
    Note that in this formulation, we omit the various connections between the
    previous state and the gates."""
    
    combined=tf.matmul(i,x_weights) +tf.matmul(o,o_weights)
    #print(combined.get_shape())
    #print(biases.get_shape())
    input_gate = tf.sigmoid(combined[:,:(num_nodes-0)]+biases[:,:(num_nodes-0)]) 
    forget_gate = tf.sigmoid(combined[:,num_nodes:(2*num_nodes-0)]+biases[:,num_nodes:(2*num_nodes-0)])
    update = combined[:,2*num_nodes:(3*num_nodes-0)]+biases[:,2*num_nodes:(3*num_nodes-0)]
    #print(forget_gate.get_shape())
    #print(input_gate.get_shape())
    #print(update.get_shape())
    state = forget_gate * state + input_gate * tf.tanh(update)
    output_gate = tf.sigmoid(combined[:,3*num_nodes:]+biases[:,3*num_nodes:])
    #print(output_gate.get_shape())
    return output_gate * tf.tanh(state), state

  # Input data.
  train_data = list()
  for _ in range(num_unrollings + 1):
    train_data.append(
      tf.placeholder(tf.float32, shape=[batch_size,vocabulary_size]))
  train_inputs = train_data[:num_unrollings]
  train_labels = train_data[1:]  # labels are inputs shifted by one time step.

  # Unrolled LSTM loop.
  outputs = list()
  output = saved_output
  state = saved_state
  for i in train_inputs:
    output, state = lstm_cell(i, output, state)
    outputs.append(output)

  # State saving across unrollings.
  # VD: tf.control_dependecies make sure that input Ops are executed before executing internal graph code. 
  # In this particul state we want to save last output and last state for next batch iteration.
  # VD: tf.Variable.assign assigns a new value to the variable.
  with tf.control_dependencies([saved_output.assign(output),
                                saved_state.assign(state)]):
    # Classifier.
    logits = tf.nn.xw_plus_b(tf.concat(0, outputs), w, b) #VD: should be similar to tf.matmul(x, w) + b 
    loss = tf.reduce_mean(
      tf.nn.softmax_cross_entropy_with_logits(
        logits, tf.concat(0, train_labels)))

  # Optimizer.
  global_step = tf.Variable(0)
  learning_rate = tf.train.exponential_decay(
    10.0, global_step, 5000, 0.1, staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate)
  gradients, v = zip(*optimizer.compute_gradients(loss))
  gradients, _ = tf.clip_by_global_norm(gradients, 1.25) # VD: clipping of the gradients to avoid explosion.
  optimizer = optimizer.apply_gradients(
    zip(gradients, v), global_step=global_step)

  # Predictions.
  train_prediction = tf.nn.softmax(logits)
  
  # Sampling and validation eval: batch 1, no unrolling.
  sample_input = tf.placeholder(tf.float32, shape=[1, vocabulary_size])
  saved_sample_output = tf.Variable(tf.zeros([1, num_nodes]))
  saved_sample_state = tf.Variable(tf.zeros([1, num_nodes]))
    # VD: tf.group creats an op that groups multiple operations.
  reset_sample_state = tf.group(
    saved_sample_output.assign(tf.zeros([1, num_nodes])),
    saved_sample_state.assign(tf.zeros([1, num_nodes])))
  sample_output, sample_state = lstm_cell(
    sample_input, saved_sample_output, saved_sample_state)
  with tf.control_dependencies([saved_sample_output.assign(sample_output),
                                saved_sample_state.assign(sample_state)]):
    sample_prediction = tf.nn.softmax(tf.nn.xw_plus_b(sample_output, w, b))

In [52]:
num_steps = 7001
summary_frequency = 100

with tf.Session(graph=graph2) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  mean_loss = 0
  for step in range(num_steps):
    batches = train_batches.next()
    feed_dict = dict()
    for i in range(num_unrollings + 1):
      feed_dict[train_data[i]] = batches[i]
    _, l, predictions, lr = session.run(
      [optimizer, loss, train_prediction, learning_rate], feed_dict=feed_dict)
    mean_loss += l
    if step % summary_frequency == 0:
      if step > 0:
        mean_loss = mean_loss / summary_frequency
      # The mean loss is an estimate of the loss over the last few batches.
      print(
        'Average loss at step %d: %f learning rate: %f' % (step, mean_loss, lr))
      mean_loss = 0
      labels = np.concatenate(list(batches)[1:])
      print('Minibatch perplexity: %.2f' % float(
        np.exp(logprob(predictions, labels))))
      if step % (summary_frequency * 10) == 0:
        # Generate some samples.
        print('=' * 80)
        for _ in range(5):
          feed = sample(random_distribution())
          sentence = characters(feed)[0]
          reset_sample_state.run()
          for _ in range(79):
            prediction = sample_prediction.eval({sample_input: feed})
            feed = sample(prediction)
            sentence += characters(feed)[0]
          print(sentence)
        print('=' * 80)
      # Measure validation set perplexity.
      reset_sample_state.run() #VD this is reset previous memory of the LSTM cell. 
                               #It means that we are involving only classifire 
      valid_logprob = 0
      for _ in range(valid_size):
        b = valid_batches.next()
        #VD: below we pass valid set consisting of two chars (e.g. "[' a']")
        predictions = sample_prediction.eval({sample_input: b[0]})
        valid_logprob = valid_logprob + logprob(predictions, b[1])
      print('Validation set perplexity: %.2f' % float(np.exp(
        valid_logprob / valid_size)))

Initialized
Average loss at step 0: 3.297403 learning rate: 10.000000
Minibatch perplexity: 27.04
u ctmfdraddn ha enojpnlv j vcrhzvl btkt ebflree shetijrfubbmeu qknno vxfghnun ea
fscbclmjl oruyb bnc  ggwag yudnzl  jeqmzsb entsnhewuwrlqid rww nlughin ruga dp r
amhwpbwxonnm ndspzq xg i e uiy  tgdfbuwged x lbk  qhcle t w erpb d lk df fepwuod
a gumn zgcivzvevtwmwnss s j tgp efsrhegdzno hoh ruzfbtl zatohajflv oivxlozsdg mc
mvsetesjilrjjpq rd j yh bbtngiuhehso aqs oklzekjdys lc at gsztoa mgsprwntmnr tmo
Validation set perplexity: 20.29
Average loss at step 100: 2.585280 learning rate: 10.000000
Minibatch perplexity: 10.05
Validation set perplexity: 10.15
Average loss at step 200: 2.243146 learning rate: 10.000000
Minibatch perplexity: 8.10
Validation set perplexity: 8.67
Average loss at step 300: 2.090176 learning rate: 10.000000
Minibatch perplexity: 7.66
Validation set perplexity: 8.16
Average loss at step 400: 2.029907 learning rate: 10.000000
Minibatch perplexity: 6.88
Validation set per

---
Problem 2
---------

We want to train a LSTM over bigrams, that is pairs of consecutive characters like 'ab' instead of single characters like 'a'. Since the number of possible bigrams is large, feeding them directly to the LSTM using 1-hot encodings will lead to a very sparse representation that is very wasteful computationally.

a- Introduce an embedding lookup on the inputs, and feed the embeddings to the LSTM cell instead of the inputs themselves.

b- Write a bigram-based LSTM, modeled on the character LSTM above.

c- Introduce Dropout. For best practices on how to use Dropout in LSTMs, refer to this [article](http://arxiv.org/abs/1409.2329).

---

---
Problem 3
---------

(difficult!)

Write a sequence-to-sequence LSTM which mirrors all the words in a sentence. For example, if your input is:

    the quick brown fox
    
the model should attempt to output:

    eht kciuq nworb xof
    
Refer to the lecture on how to put together a sequence-to-sequence model, as well as [this article](http://arxiv.org/abs/1409.3215) for best practices.

---